In [24]:
from data_preprocess import data_preprocess, get_input_data, get_training_data

data = data_preprocess(one_hot_location=True)

features = ['absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
       'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
       'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
       'direct_rad_1h:J', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'is_day:idx',
       'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
       'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2',
       'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'wind_speed_w_1000hPa:ms', 'A', 'B', 'C', 'time']

X_train, targets = get_training_data(data, features)

/Users/andreastallvik/Projects/tdt4173-project/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [35]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

cat_cols = ['is_day:idx', 'is_in_shadow:idx','month', 'hour', 'day']

one_hot_cols = ['A', 'B', 'C']

num_cols = ['absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
       'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
       'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
       'direct_rad_1h:J', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm','msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
       'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2',
       'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'wind_speed_w_1000hPa:ms']


column_trans = ColumnTransformer(
    [('categories', OneHotEncoder(dtype='int'), cat_cols),
     ('numerical', MinMaxScaler(), num_cols),
     ('one_hot_allready', 'passthrough', one_hot_cols),],
     remainder='drop', verbose_feature_names_out=False)

In [36]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor

class TimeFeatureAdder(BaseEstimator, TransformerMixin):
    """Adds the features month and hour to the data"""

    def __init__(self, add_features=True):
        self.add_features = add_features

    def fit(self, X, y=None):
        return self

    def transform(self, X):

        X_copy = X.copy()

        timestamps = X["time"]
        month = timestamps.apply(lambda x: x.month)
        hour = timestamps.apply(lambda x: x.hour)
        day = timestamps.apply(lambda x: x.day)

        if self.add_features:
            X_copy["month"] = month
            X_copy["hour"] = hour
            X_copy["day"] = day
            return X_copy
        else:
            return X_copy
        
class ColumnDropper(BaseEstimator, TransformerMixin):
    """Drops the time column from the data"""

    def __init__(self, drop_time=True, drop_cols = []):
        self.drop_time = drop_time
        self.drop_cols = drop_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):

        X_copy = X.copy()

        if self.drop_time:
            return X_copy.drop(columns=['time'])

        X_copy.drop(columns=self.drop_cols)

        return X_copy

data_process_pipeline = Pipeline([
    ('add_features', TimeFeatureAdder()),
    ('drop_cols', ColumnDropper()),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
    ('minMaxScaling', MinMaxScaler()),
])

data_process_2 = Pipeline([
    ('add_features', TimeFeatureAdder()),
    #('drop_cols', ColumnDropper()),
    ('column_transform', column_trans),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
])

# Training classifiers
# reg1 = GradientBoostingRegressor(random_state=1)
# reg2 = RandomForestRegressor(random_state=1)
# reg3 = LinearRegression()

whole_model_pipeline = Pipeline([
    #('data_process', data_process_pipeline),
    ('data_process', data_process_2),
    ('random_forest', RandomForestRegressor(random_state=3)),
    #('voting_regressor', VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)]))
])

In [37]:
whole_model_pipeline.fit(X_train, targets)

Pipeline(steps=[('data_process',
                 Pipeline(steps=[('add_features', TimeFeatureAdder()),
                                 ('column_transform',
                                  ColumnTransformer(transformers=[('categories',
                                                                   OneHotEncoder(dtype='int'),
                                                                   ['is_day:idx',
                                                                    'month',
                                                                    'hour',
                                                                    'day', 'A',
                                                                    'B', 'C']),
                                                                  ('numerical',
                                                                   MinMaxScaler(),
                                                                   ['absolute_humidity_2m:gm3',
                                                                    'air_density_2m:kgm3',
                                                                    'ceiling_height_agl:m',
                                                                    'clear_sky_energy_1h:J'...
                                                                    'super_cooled_liquid_water:kgm2',
                                                                    't_1000hPa:K',
                                                                    'total_cloud_cover:p',
                                                                    'visibility:m',
                                                                    'wind_speed_10m:ms',
                                                                    'wind_speed_u_10m:ms',
                                                                    'wind_speed_v_10m:ms', ...]),
                                                                  ('one_hot_allready',
                                                                   'passthrough',
                                                                   ['A'])],
                                                    verbose_feature_names_out=False)),
                                 ('imputer',
                                  SimpleImputer(fill_value=0,
                                                strategy='constant'))])),
                ('random_forest', RandomForestRegressor(random_state=3))])

In [38]:
from data_preprocess import prepare_submission

X_train, targets = get_training_data(data, features)

X_test = get_input_data()
predictions = whole_model_pipeline.predict(X_test[features])

submission = prepare_submission(X_test, predictions)

submission.to_csv('rf_one_hot.csv', index=False)

/Users/andreastallvik/Projects/tdt4173-project/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/andreastallvik/Projects/tdt4173-project/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [29]:
from evaluate import mean_squared_error

mean_squared_error(targets, whole_model_pipeline.predict(X_train))

9426.48475739214

In [ ]:
drop_cols = ['precip_type_5min:idx',
  'is_in_shadow:idx',
  'rain_water:kgm2',
  'B',
  'fresh_snow_12h:cm',
  'C',
  'fresh_snow_6h:cm',
  'snow_depth:cm',
  'snow_density:kgm3',
  'snow_melt_10min:mm',
  'prob_rime:p',
  'fresh_snow_3h:cm',
  'dew_or_rime:idx',
  'wind_speed_w_1000hPa:ms',
  'fresh_snow_1h:cm',
  'snow_drift:idx']

In [85]:
#chatgpt on using gridsearch to find the best params

from sklearn.model_selection import GridSearchCV

# Define the parameters you want to search
param_grid = {
    'data_process__drop_cols__drop_cols': [["precip_type_5min:idx", "is_in_shadow:idx", "rain_water:kgm2", "B", "fresh_snow_12h:cm", "C", "fresh_snow_6h:cm",
             "snow_depth:cm", "snow_density:kgm3", "snow_melt_10min:mm", "prob_rime:p", "fresh_snow_3h:cm", "dew_or_rime:idx",
             "wind_speed_w_1000hPa:ms", "fresh_snow_1h:cm", "snow_drift:idx"], ["precip_type_5min:idx", "rain_water:kgm2", "B", "fresh_snow_12h:cm", "C", "fresh_snow_6h:cm", "prob_rime:p", "fresh_snow_3h:cm", "dew_or_rime:idx",
             "wind_speed_w_1000hPa:ms", "fresh_snow_1h:cm", "snow_drift:idx"], ['direct_rad_1h:J', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'is_day:idx',
       'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p','sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',]]
}

# Create a GridSearchCV instance
grid_search = GridSearchCV(estimator=whole_model_pipeline, param_grid=param_grid, cv=5)

# Perform the grid search on your data
grid_search.fit(X_train, targets)

# Get the best parameters and best estimator from the grid search
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Fit the best estimator on your data
best_estimator.fit(X_train, targets)

Pipeline(steps=[('data_process',
                 Pipeline(steps=[('add_features', TimeFeatureAdder()),
                                 ('drop_cols',
                                  ColumnDropper(drop_cols=['precip_type_5min:idx',
                                                           'is_in_shadow:idx',
                                                           'rain_water:kgm2',
                                                           'B',
                                                           'fresh_snow_12h:cm',
                                                           'C',
                                                           'fresh_snow_6h:cm',
                                                           'snow_depth:cm',
                                                           'snow_density:kgm3',
                                                           'snow_melt_10min:mm',
                                                           'prob_rime:p',
                                                           'fresh_snow_3h:cm',
                                                           'dew_or_rime:idx',
                                                           'wind_speed_w_1000hPa:ms',
                                                           'fresh_snow_1h:cm',
                                                           'snow_drift:idx'])),
                                 ('imputer',
                                  SimpleImputer(fill_value=0,
                                                strategy='constant')),
                                 ('minMaxScaling', MinMaxScaler())])),
                ('random_forest', RandomForestRegressor(random_state=3))])

In [86]:
best_params

{'data_process__drop_cols__drop_cols': ['precip_type_5min:idx',
  'is_in_shadow:idx',
  'rain_water:kgm2',
  'B',
  'fresh_snow_12h:cm',
  'C',
  'fresh_snow_6h:cm',
  'snow_depth:cm',
  'snow_density:kgm3',
  'snow_melt_10min:mm',
  'prob_rime:p',
  'fresh_snow_3h:cm',
  'dew_or_rime:idx',
  'wind_speed_w_1000hPa:ms',
  'fresh_snow_1h:cm',
  'snow_drift:idx']}

In [87]:
best_estimator

Pipeline(steps=[('data_process',
                 Pipeline(steps=[('add_features', TimeFeatureAdder()),
                                 ('drop_cols',
                                  ColumnDropper(drop_cols=['precip_type_5min:idx',
                                                           'is_in_shadow:idx',
                                                           'rain_water:kgm2',
                                                           'B',
                                                           'fresh_snow_12h:cm',
                                                           'C',
                                                           'fresh_snow_6h:cm',
                                                           'snow_depth:cm',
                                                           'snow_density:kgm3',
                                                           'snow_melt_10min:mm',
                                                           'prob_rime:p',
                                                           'fresh_snow_3h:cm',
                                                           'dew_or_rime:idx',
                                                           'wind_speed_w_1000hPa:ms',
                                                           'fresh_snow_1h:cm',
                                                           'snow_drift:idx'])),
                                 ('imputer',
                                  SimpleImputer(fill_value=0,
                                                strategy='constant')),
                                 ('minMaxScaling', MinMaxScaler())])),
                ('random_forest', RandomForestRegressor(random_state=3))])